In [ ]:
from temporal_grounding.temporal_grounding import extract_timestamps_from_video

vp = r"testing\shot_retrevial\videos\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.avi"
sp = r"testing\shot_retrevial\timestamps\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.json"
extract_timestamps_from_video(vp, sp)

In [ ]:
from temporal_grounding.temporal_grounding import convert_float_to_time_str, convert_time_to_float
from data_prep import *
from datascience import *
import numpy as np

log_path = get_log_path("972173")
log_path

start_time = 600
# convert_float_to_time_str(531.199)
convert_time_to_float("5:13")

# arr = []
# with open(log_path) as file:
#     doc = csv.reader(file, delimiter=';')
#     for row in doc:
#         arr.append(row)

# for row in arr[1: ]:
#     if row[2][0] == "2" or row[2][0] == "3":
#         print("Shot", row[26], "at", row[23])


In [ ]:
import cv2

def clip_video(input_path, output_path, start_frame, end_frame):
    try:
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise Exception("Error: Could not open video file")
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
            if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
                break
        cap.release()
        out.release()
    except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
from data_prep import *
import os
import json

video_path = r"testing\shot_retrevial\videos\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.avi"
shots_out_path = r"testing\shot_retrevial\shot_clips"
timestamps_path = r"testing\shot_retrevial\timestamps\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.json"

def extract_shots_from_video(video_path, shots_out_dir, timestamps_path=None):

    try:
        game_id = os.path.basename(video_path).split("_")[0]
    except:
        raise Exception(f"Bad video path: {video_path}")
    try:
        with open(timestamps_path, "r") as f:
            timestamps = json.load(f)
    except:
        raise Exception(f"Error: bad timestamps path: {timestamps_path}.")

    path_to_logs = get_log_path(game_id)
    shot_events = get_shot_events(path_to_logs)
    intervals = []
    for event in shot_events:
        key = f"{event[2]}_{event[3]}"
        if key in timestamps:
            try:
                backwards_pad = 15
                shot_duration = 30 * 3
                start_frame = timestamps[key] - backwards_pad
                end_frame = start_frame + shot_duration
                intervals.append([start_frame, end_frame, event])
            except:
                pass

    shot_index = 0
    video_title = os.path.basename(video_path)
    shot_subdir = os.path.join(shots_out_dir, video_title.split(".")[0])
    os.makedirs(shot_subdir, exist_ok=True)
    for interval in intervals:
        start_frame, end_frame = interval[0], interval[1]
        is_shot_made = interval[2][1] 
        output_path = os.path.join(shot_subdir, f"_clip_{shot_index}_{is_shot_made}_.avi")
        print(f"Shot clips saved to: {output_path}")
        clip_video(video_path, output_path, start_frame, end_frame)
        shot_index += 1
        

extract_shots_from_video(video_path, shots_out_path, timestamps_path)

In [ ]:
( 6 * 60 ) + 30

In [1]:
from datascience import *
import csv
import pandas as pd
from data_prep import get_log_path
from temporal_grounding.temporal_grounding import convert_float_to_time_str
import numpy as np

id = "936399"
path = get_log_path(id)

In [2]:
df = pd.read_csv(path, delimiter=';', skiprows=1)

In [54]:
def build_table_from_game_log(game_id, quarter):

    t = Table.from_df(df)
    columns = [str(i) for i in range(31)]
    labels = np.array(t.labels)
    t = t.relabel(labels, columns).select("2", "13", "22", "23")
    t = t.relabel("2", "event")
    t = t.relabel("22", "time_elapsed")
    t = t.relabel("13", "quarter")
    t = t.relabel("23", "time_elapsed_high")
    start_time = int(max(t.column("time_elapsed_high")))
    time_elapsed_col = t.column("time_elapsed")
    time_remaining_col = []
    for val in time_elapsed_col:
        time = start_time-val
        time_remaining_col.append(
            convert_float_to_time_str(time))
    t = t.with_column("time_remaining", time_remaining_col)
    t = t.where("event", (are.containing("2") | are.containing("3"))
                & are.not_containing("F"))
    t = t.where("quarter", are.equal_to(quarter))
    t = t.move_column("time_remaining", 2)
    return t




In [55]:
id = "936399"
build_table_from_game_log(id, 3)


event,quarter,time_remaining,time_elapsed,time_elapsed_high
2+,3,10:00.0,0,11.356
2-,3,09:45.5,14.425,28.92
2+,3,09:29.1,30.81,45.676
2-,3,09:29.1,30.81,45.676
2+,3,08:54.3,65.605,81.82
3-,3,08:32.1,87.854,101.447
3-,3,08:10.1,109.801,125.181
2-,3,07:51.7,128.292,128.404
3-,3,07:50.4,129.514,135.891
3+,3,07:41.5,138.455,154.124


In [43]:
t = Table.from_df(df)
columns = [str(i) for i in range(31)]
labels = np.array(t.labels)
t = t.relabel(labels, columns).select("2", "13", "22", "23")
t = t.relabel("2", "event")
t = t.relabel("22", "time_elapsed")
t = t.relabel("13", "quarter")
t = t.relabel("23", "time_elapsed_high")
start_time = int(max(t.column("time_elapsed_high")))
time_elapsed_col = t.column("time_elapsed")
time_remaining_col = []
for val in time_elapsed_col:
    time = start_time-val
    time_remaining_col.append(
        convert_float_to_time_str(time))
t = t.with_column("time_remaining", time_remaining_col)
t = t.where("event", (are.containing("2") | are.containing("3"))
            & are.not_containing("F"))
t

event,quarter,time_elapsed,time_elapsed_high,time_remaining
2-,1,22.38,39.944,09:37.6
3+,1,50.436,56.67,09:09.5
2+,1,61.134,78.85,08:58.8
2-,1,81.809,93.614,08:38.1
3-,1,95.134,100.375,08:24.8
2+,1,102.754,116.381,08:17.2
2-,1,131.48,137.649,07:48.5
3-,1,150.106,150.836,07:29.8
2+,1,153.406,164.006,07:26.5
2+,1,169.446,189.787,07:10.5
